# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID               City      Lat       Lng  Max Temp  Humidity  \
0        0  sao joao da barra -21.6403  -41.0511     21.44        91   
1        1      blackmans bay -43.0167  147.3167     15.94        49   
2        2     chernyshevskiy  63.0128  112.4714      1.82        73   
3        3               okha  53.5739  142.9478     17.21        76   
4        4           ust-nera  64.5667  143.2000      7.80        75   

   Cloudiness  Wind Speed Country        Date  
0          57        4.25      BR  1694136336  
1           2        2.55      AU  1694136496  
2         100        2.90      RU  1694136497  
3         100        8.41      RU  1694136497  
4         100        1.93      RU  1694136498

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot

matplot_1 = city_data_df.hvplot.points("Lng","Lat",geo=True,size="Humidity",color="City",tiles="OSM",
                           alpha=0.5,frame_width=700,frame_height=500)
# Display the map
matplot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
idea_city_df = city_data_df[(city_data_df["Max Temp"]>25) & (city_data_df["Humidity"] < 60) \
                           & (city_data_df["Max Temp"]<30)].dropna()

# Drop any rows with null values


# Display sample data
idea_city_df.head()

City_ID               City      Lat       Lng  Max Temp  Humidity  \
20        20               kuna  43.4918 -116.4201     29.45        36   
33        33             labasa -16.4167  179.3833     27.63        48   
39        39    charters towers -20.1000  146.2667     28.78        45   
126      126  placido de castro -10.3353  -67.1856     28.75        32   
140      140              feijo  -8.1642  -70.3536     27.59        41   

     Cloudiness  Wind Speed Country        Date  
20            0        3.09      US  1694136504  
33           68        2.76      FJ  1694136509  
39            8        3.10      AU  1694136511  
126           5        0.63      BR  1694136548  
140          29        0.32      BR  1694136554

### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idea_city_df[["City","Country","Lat","Lng","Humidity"]].copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=''

# Display sample data
hotel_df.head()

City Country      Lat       Lng  Humidity Hotel Name
20                kuna      US  43.4918 -116.4201        36           
33              labasa      FJ -16.4167  179.3833        48           
39     charters towers      AU -20.1000  146.2667        45           
126  placido de castro      BR -10.3353  -67.1856        32           
140              feijo      BR  -8.1642  -70.3536        41

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit":20  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kuna - nearest hotel: No hotel found
labasa - nearest hotel: Grand Eastern Hotel
charters towers - nearest hotel: No hotel found
placido de castro - nearest hotel: No hotel found
feijo - nearest hotel: Hotel JOAFRAN
east ballina - nearest hotel: Ballina Beach Resort
brezina - nearest hotel: No hotel found
nganjuk - nearest hotel: Hotel Nirwana
kununurra - nearest hotel: Kununurra Country Club Resort
cacu - nearest hotel: No hotel found
winnemucca - nearest hotel: The Overland Hotel
kot radha kishan - nearest hotel: No hotel found
presidente medici - nearest hotel: No hotel found
murzuq - nearest hotel: فندق باريس - مرزق
vernon - nearest hotel: Okanagan Royal Park Inn
karratha - nearest hotel: Karratha International Hotel
tuba city - nearest hotel: Quality Inn Navajo Nation
ghat - nearest hotel: فندق تاسيلي
tubarjal - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
benxi - nearest hotel: 富佳大酒店
altamira - nearest hotel: Hotel Vale do Xingu
san

City Country      Lat       Lng  Humidity  \
20                        kuna      US  43.4918 -116.4201        36   
33                      labasa      FJ -16.4167  179.3833        48   
39             charters towers      AU -20.1000  146.2667        45   
126          placido de castro      BR -10.3353  -67.1856        32   
140                      feijo      BR  -8.1642  -70.3536        41   
142               east ballina      AU -28.8667  153.5833        56   
216                    brezina      DZ  33.0989    1.2608        34   
228                    nganjuk      ID  -7.6051  111.9035        56   
254                  kununurra      AU -15.7667  128.7333        48   
320                       cacu      BR -18.5567  -51.1308        32   
343                 winnemucca      US  40.9730 -117.7357        17   
368           kot radha kishan      PK  31.1725   74.0997        55   
401          presidente medici      BR -11.1753  -61.9014        43   
403                     murzuq      LY  25.9155   13.9184        23   
407                     vernon      CA  50.2581 -119.2691        34   
458                   karratha      AU -20.7377  116.8463        20   
461                  tuba city      US  36.1350 -111.2399        25   
462                       ghat      LY  24.9647   10.1728        28   
474                   tubarjal      SA  30.4999   38.2160        25   
488                tamanrasset      DZ  22.7850    5.5228        26   
509                      benxi      CN  41.2886  123.7650        52   
513                   altamira      BR  -3.2033  -52.2064        55   
521  santo antonio do leverger      BR -15.8656  -56.0767        44   
532                      kifri      IQ  34.6959   44.9661        20   
552                     makary      CM  12.5717   14.4666        52   

                        Hotel Name  
20                  No hotel found  
33             Grand Eastern Hotel  
39                  No hotel found  
126                 No hotel found  
140                  Hotel JOAFRAN  
142           Ballina Beach Resort  
216                 No hotel found  
228                  Hotel Nirwana  
254  Kununurra Country Club Resort  
320                 No hotel found  
343             The Overland Hotel  
368                 No hotel found  
401                 No hotel found  
403              فندق باريس - مرزق  
407        Okanagan Royal Park Inn  
458   Karratha International Hotel  
461      Quality Inn Navajo Nation  
462                    فندق تاسيلي  
474                 No hotel found  
488                    فندق الأمان  
509                          富佳大酒店  
513            Hotel Vale do Xingu  
521                 No hotel found  
532                 No hotel found  
552                 No hotel found

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points("Lng",
                                 "Lat",
                                 geo=True,
                                 tiles="OSM",
                                 frame_width=800,
                                 frame_height=600,
                                 scale=0.5,
                                alpha=0.5,
                                 color="City",
                                 hover_cols = ["Hotel Name","Country"]
                                 )

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)